### Imports

In [1]:
import os
import re
from typing import Dict

import mne
import numpy as np
import scipy.stats as sp_stats
from joblib import Parallel, delayed
from NirsLabProject.config.paths import Paths

from NirsLabProject.utils import pipeline_utils as putils
from NirsLabProject.config.consts import *

from NirsLabProject.utils import general_utils as utils
from NirsLabProject.config.consts import *
from NirsLabProject.config.subject import Subject
from NirsLabProject.utils.group_spikes import group_spikes
from NirsLabProject.utils import plotting


### Extract Paths of stimuli features
These are the features from the raw edf data without grouping, stimuli and sleeping features

In [2]:
subjects = [Subject(d, True) for d in os.listdir(Paths.products_data_dir_path) if d.startswith('p')]
stimuli_subjects = filter(lambda subj: subj.stimuli_project, subjects)
stimuli_subjects_with_features = filter(lambda subj: os.path.exists(subj.paths.subject_channels_spikes_features_path), stimuli_subjects)
stimuli_subjects_with_features = sorted(stimuli_subjects_with_features, key=lambda s: s.p_number)

In [4]:
def edit_coordinates(channels_spikes_features):
    channel_name_to_coordinates = utils.calculate_coordinates(subj)
    for channel, data in channels_spikes_features.items():
        x, y, z = utils.get_coordinates_of_channel(channel_name_to_coordinates, channel)
        data[:, CORD_X_INDEX] = x
        data[:, CORD_Y_INDEX] = y
        data[:, CORD_Z_INDEX] = z
    return channels_spikes_features

def regenerate_flat_features(subj: Subject):
    channels_spikes_features = np.load(
        subj.paths.subject_channels_spikes_features_path,
        allow_pickle=True
    ).item()
    channels_spikes_features = edit_coordinates(channels_spikes_features)
    index_to_channel = putils.get_index_to_channel(subj, channels_spikes_features)
    groups, flat_features = putils.group_spikes(subj, channels_spikes_features, index_to_channel)
    flat_features = utils.add_flag_of_scalp_detection_to_spikes_features(flat_features, None)
    flat_features = utils.add_stimuli_flag_to_spikes_features(subj, flat_features)
    flat_features = utils.add_sleeping_stage_flag_to_spike_features(subj, flat_features)
    subject_id = np.ones((flat_features.shape[0], 1)) * subj.p_number
    flat_features = np.concatenate((flat_features, subject_id), axis=1)
    return flat_features

for subj in stimuli_subjects_with_features:
    flat_features = regenerate_flat_features(subj)
    np.save(subj.paths.subject_flat_features_path, flat_features)
    

calculating coordinates for subject 485
Grouping spikes
calculating coordinates for subject 486
channel C3 not found in coordinates file
Grouping spikes
calculating coordinates for subject 487
channel LSTG5 not found in coordinates file
channel LSTG6 not found in coordinates file
Grouping spikes
calculating coordinates for subject 488
Grouping spikes
calculating coordinates for subject 489
channel C3 not found in coordinates file
channel LSTG6 not found in coordinates file
channel RSTG5 not found in coordinates file
Grouping spikes
calculating coordinates for subject 496
channel C3 not found in coordinates file
Grouping spikes
calculating coordinates for subject 497
Grouping spikes
calculating coordinates for subject 498
channel LOF7 not found in coordinates file
Grouping spikes
calculating coordinates for subject 499
channel RAI6 not found in coordinates file
channel RSTG4 not found in coordinates file
channel RSTG5 not found in coordinates file
Grouping spikes
calculating coordinates

In [9]:
import shutil
import os

for subj in stimuli_subjects_with_features:
    new_path = '/tmp/flat_features'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    shutil.copyfile(subj.paths.subject_flat_features_path, os.path.join(new_path, f'{subj.p_number}_flat_features.npy'))